In [2]:
import requests
import pandas as pd
from itertools import cycle
from datetime import date 
import numpy as np
import random
import time
import collections
import warnings

In [3]:
# df = pd.read_csv("Fundamental_Data.csv",index_col=0)
df = pd.read_csv("C:/Users/LEGION/Downloads/Fundamental_Data.csv", index_col = 0)
# df = pd.read_csv("C:/Projects/Automated-Fundamental-Analysis-master/Automated-Fundamental-Analysis-master/Fundamental_Data.csv", index_col = 0)
df.head()

,Ticker,Company,Market Cap,Sector,Industry,Country,Fwd P/E,PEG,P/S,P/B,...,Volatility M,SMA20,SMA50,SMA200,52W High,52W Low,RSI,Earnings,Price,Target Price
0,A,"Agilent Technologies, Inc.",39.96B,Healthcare,Diagnostics & Research,USA,25.16,2.28,6.20,7.87,...,2.74%,0.52%,-0.01%,-10.76%,-24.70%,9.87%,50.18,Feb 22/a,135.21,164.44
1,AA,Alcoa Corporation,16.65B,Basic Materials,Aluminum,USA,9.12,-,1.37,3.56,...,5.93%,1.17%,13.08%,59.75%,-11.41%,189.43%,52.96,Apr 20/a,86.90,86.42
2,AAAU,Goldman Sachs Physical Gold ETF,-,Financial,Exchange Traded Fund,USA,-,-,-,-,...,1.12%,-1.22%,1.35%,5.65%,-7.05%,11.89%,49.04,-,19.12,-
3,AAC,Ares Acquisition Corporation,1.23B,Financial,Shell Companies,USA,-,-,-,1.32,...,0.24%,0.58%,0.76%,0.69%,-1.70%,1.76%,71.73,-,9.83,-
4,AACG,ATA Creativity Global,45.53M,Consumer Defensive,Education & Training Services,China,-,-,1.43,1.52,...,9.54%,3.57%,-1.97%,-34.16%,-70.53%,40.00%,50.61,Mar 17/a,1.40,32.00


In [4]:
df.columns

Index(['Ticker', 'Company', 'Market Cap', 'Sector', 'Industry', 'Country',
       'Fwd P/E', 'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Dividend',
       'Payout Ratio', 'EPS this Y', 'EPS next Y', 'EPS past 5Y',
       'EPS next 5Y', 'Sales past 5Y', 'EPS Q/Q', 'Sales Q/Q', 'Insider Own',
       'Insider Trans', 'Inst Own', 'Inst Trans', 'Short Ratio', 'ROA', 'ROE',
       'ROI', 'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M', 'Oper M',
       'Profit M', 'Perf Month', 'Perf Quart', 'Perf Half', 'Perf Year',
       'Perf YTD', 'Volatility M', 'SMA20', 'SMA50', 'SMA200', '52W High',
       '52W Low', 'RSI', 'Earnings', 'Price', 'Target Price'],
      dtype='object')

In [5]:
# df.drop(['Overall Rating', 'Valuation Grade', 'Profitability Grade', 'Growth Grade', 'Performance Grade', 'Percent Diff'], axis = 1, inplace=True)
# # df.drop(['Valuation Grade', 'Profitability Grade', 'Growth Grade', 'Performance Grade'], axis = 1, inplace=True)
# df.to_csv("Fundamental_Data.csv")

In [6]:
# df.columns

In [7]:
sector_data = collections.defaultdict(lambda : collections.defaultdict(dict))
sector_data

defaultdict(<function __main__.<lambda>()>, {})

In [8]:
def remove_outliers(S, std):    
    s1 = S[~((S-S.mean()).abs() > std * S.std())]
    return s1[~((s1-s1.mean()).abs() > std * s1.std())]

In [9]:
metrics = df.columns[6: -3]
metrics

Index(['Fwd P/E', 'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Dividend',
       'Payout Ratio', 'EPS this Y', 'EPS next Y', 'EPS past 5Y',
       'EPS next 5Y', 'Sales past 5Y', 'EPS Q/Q', 'Sales Q/Q', 'Insider Own',
       'Insider Trans', 'Inst Own', 'Inst Trans', 'Short Ratio', 'ROA', 'ROE',
       'ROI', 'Curr R', 'Quick R', 'LTDebt/Eq', 'Debt/Eq', 'Gross M', 'Oper M',
       'Profit M', 'Perf Month', 'Perf Quart', 'Perf Half', 'Perf Year',
       'Perf YTD', 'Volatility M', 'SMA20', 'SMA50', 'SMA200', '52W High',
       '52W Low', 'RSI'],
      dtype='object')

In [10]:
sectors = df['Sector'].unique()
metrics = df.columns[6: -3]


for sector in sectors:
    rows = df.loc[df['Sector'] == sector]
    for metric in metrics:
        rows[metric] = rows[metric].str.rstrip('%')
        rows[metric] = pd.to_numeric(rows[metric], errors='coerce')
        data = remove_outliers(rows[metric], 2)
        
        sector_data[sector][metric]['Median'] = data.median(skipna=True)
        sector_data[sector][metric]['10Pct'] = data.quantile(0.1)
        sector_data[sector][metric]['90Pct'] = data.quantile(0.9)
        sector_data[sector][metric]['Std'] = np.std(data, axis=0) / 5

C:\Users\LEGION\AppData\Local\Temp\ipykernel_16456\53926490.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[metric] = rows[metric].str.rstrip('%')
C:\Users\LEGION\AppData\Local\Temp\ipykernel_16456\53926490.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[metric] = pd.to_numeric(rows[metric], errors='coerce')
C:\Users\LEGION\AppData\Local\Temp\ipykernel_16456\53926490.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [11]:
data_to_add = collections.defaultdict(list)
data_to_add

defaultdict(list, {})

In [12]:
def get_metric_grade(sector, metric_name, metric_val):
    lessThan = metric_name in ['Fwd P/E', 'PEG', 'P/S', 'P/B', 'P/FCF', 'Volatility M']
    grade_basis = '10Pct' if lessThan else '90Pct'
    start, change = sector_data[sector][metric_name][grade_basis], sector_data[sector][metric_name]['Std']
    
    grade_map = {'A+': 0, 'A': change, 'A-' : change * 2, 'B+' : change * 3, 'B' : change * 4, 
                 'B-' : change * 5, 'C+' : change * 6, 'C' : change * 7, 'C-' : change * 8, 
                 'D+' : change * 9, 'D' : change * 10, 'D-' : change * 11, 'F' : change * 12
            }
    
    for grade, val in grade_map.items():
        comparison = start + val if lessThan else start - val
        if lessThan and metric_val < comparison:
            return grade
        if lessThan == False and metric_val > comparison:
            return grade
    return 'C'

In [13]:
def get_metric_val(ticker, metric_name):
    try:
        return float(str(df.loc[df['Ticker'] == ticker][metric_name].values[0]).rstrip("%"))
    except:
        return 0

In [14]:
grading_metrics = {'Valuation' : ['Fwd P/E', 'PEG', 'P/S', 'P/B', 'P/FCF'],
                    'Profitability' : ['Profit M', 'Oper M', 'Gross M', 'ROE', 'ROA'],
                    'Growth' : ['EPS this Y', 'EPS next Y', 'EPS next 5Y', 'Sales Q/Q', 'EPS Q/Q'],
                    'Performance' : ['Perf Month', 'Perf Quart', 'Perf Half', 'Perf Year', 'Perf YTD', 'Volatility M']
                }

grade_scores = {'A+' : 4.3, 'A' : 4.0, 'A-' : 3.7, 'B+' : 3.3, 'B' : 3.0, 'B-' : 2.7, 
                'C+' : 2.3, 'C' : 2.0, 'C-' : 1.7, 'D+' : 1.3, 'D' : 1.0, 'D-' : 0.7, 'F' : 0.0
            }

def get_category_grades(ticker, sector):
    category_grades = {}
    for category in grading_metrics:
        metric_grades = []
        for metric_name in grading_metrics[category]:
            metric_grades.append(get_metric_grade(sector, metric_name, get_metric_val(ticker, metric_name)))
        category_grades[category] = metric_grades
        
    for category in category_grades:
        score = 0
        for grade in category_grades[category]:
            score += grade_scores[grade]
        category_grades[category].append(round(score / len(category_grades[category]), 2))
    return category_grades

In [15]:
def get_stock_rating(category_grades):
    score = 0
    for category in category_grades:
        score += category_grades[category][-1]
    return round(score * 6.2, 2)

In [16]:
def convert_to_letter_grade(val):
    for grade in grade_scores:
        if val >= grade_scores[grade]:
            return grade

In [17]:
for row in df.iterrows():
    ticker, sector = row[1]['Ticker'], row[1]['Sector']
    category_grades = get_category_grades(ticker, sector)
    stock_rating = get_stock_rating(category_grades)
    
    data_to_add['Overall Rating'].append(stock_rating)
    data_to_add['Valuation Grade'].append(convert_to_letter_grade(category_grades['Valuation'][-1]))
    data_to_add['Profitability Grade'].append(convert_to_letter_grade(category_grades['Profitability'][-1]))
    data_to_add['Growth Grade'].append(convert_to_letter_grade(category_grades['Growth'][-1]))
    data_to_add['Performance Grade'].append(convert_to_letter_grade(category_grades['Performance'][-1]))

In [18]:
df['Overall Rating'] = data_to_add['Overall Rating']
df['Valuation Grade'] = data_to_add['Valuation Grade']
df['Profitability Grade'] = data_to_add['Profitability Grade']
df['Growth Grade'] = data_to_add['Growth Grade']
df['Performance Grade'] = data_to_add['Performance Grade']    
df['Percent Diff'] = (pd.to_numeric(df['Target Price'], errors='coerce') - pd.to_numeric(df['Price'], errors='coerce')) / pd.to_numeric(df['Price'], errors='coerce') * 100

In [19]:
df.head()

,Ticker,Company,Market Cap,Sector,Industry,Country,Fwd P/E,PEG,P/S,P/B,...,RSI,Earnings,Price,Target Price,Overall Rating,Valuation Grade,Profitability Grade,Growth Grade,Performance Grade,Percent Diff
0,A,"Agilent Technologies, Inc.",39.96B,Healthcare,Diagnostics & Research,USA,25.16,2.28,6.20,7.87,...,50.18,Feb 22/a,135.21,164.44,74.40,B,B+,C+,B-,21.618224
1,AA,Alcoa Corporation,16.65B,Basic Materials,Aluminum,USA,9.12,-,1.37,3.56,...,52.96,Apr 20/a,86.90,86.42,63.92,B,D+,C,B+,-0.552359
2,AAAU,Goldman Sachs Physical Gold ETF,-,Financial,Exchange Traded Fund,USA,-,-,-,-,...,49.04,-,19.12,-,66.96,A+,C-,D+,B,NaN
3,AAC,Ares Acquisition Corporation,1.23B,Financial,Shell Companies,USA,-,-,-,1.32,...,71.73,-,9.83,-,62.99,A,D+,D+,B-,NaN
4,AACG,ATA Creativity Global,45.53M,Consumer Defensive,Education & Training Services,China,-,-,1.43,1.52,...,50.61,Mar 17/a,1.40,32.00,73.22,A-,C-,B,B-,2185.714286


In [20]:
# df_rel = df[["Company","Sector","Industry","Overall Rating","Valuation Grade","Profitability Grade", "Growth Grade","Performance Grade","Percent Diff"]]

In [21]:
# df_rel["Sector"].value_counts() 
#11 sectors and 149 industries

In [22]:
# df_rel.groupby(by="Sector").head()